# Extreme Event Analysis in ERA5 PV Variables


In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import sys
import os


# Add the project root to sys.path to allow imports from scripts/
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from scripts.analysis import extreme_events



# Path to the merged NetCDF file
merged_file = os.path.join(project_root, 'data', 'processed', 'era5_merged_Bonn.nc')

# Load the merged dataset
try:
    ds = xr.open_dataset(merged_file)
    print(f"Successfully loaded merged dataset from {merged_file}")
    print("\nDataset summary:")
    print(ds)
except Exception as e:
    print(f"Error loading merged dataset: {e}")
    raise

df = ds.to_dataframe().reset_index()



/var/folders/z6/6d6lkl057p12xjjzc2ml33vh0000gn/T/ipykernel_11204/3295624429.py:26: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  ds = xr.open_dataset(merged_file)


Successfully loaded merged dataset from /Users/iman/work/ERA5_basic_energy_analysis/data/processed/era5_merged_Bonn.nc

Dataset summary:
<xarray.Dataset> Size: 351kB
Dimensions:    (time: 5844, latitude: 1, longitude: 1)
Coordinates:
  * time       (time) datetime64[ns] 47kB 2021-01-01 ... 2024-12-31T18:00:00
  * latitude   (latitude) float64 8B 50.7
  * longitude  (longitude) float64 8B 7.0
    number     int64 8B ...
    step       timedelta64[ns] 8B ...
    surface    float64 8B ...
Data variables: (12/13)
    sp         (time, latitude, longitude) float32 23kB ...
    strd       (time, latitude, longitude) float32 23kB ...
    v10        (time, latitude, longitude) float32 23kB ...
    ssrd       (time, latitude, longitude) float32 23kB ...
    cbh        (time, latitude, longitude) float32 23kB ...
    tcc        (time, latitude, longitude) float32 23kB ...
    ...         ...
    tsr        (time, latitude, longitude) float32 23kB ...
    str        (time, latitude, longitude) fl

## Prolonged Low Irradiance Events

In [11]:
# ===== CONFIGURATION =====
# Radiation thresholds (W/m²)
THRESHOLD = 120          # W/m² (12% of peak capacity)
MIN_CONSECUTIVE = 2      # Minimum consecutive 6h periods (12 hours)
PEAK_HOURS = (10, 14)    # Peak production hours (10 AM - 2 PM)
MIN_CONSECUTIVE_DAYS = 3  # Minimum consecutive days for significant event

# Time parameters
TIME_STEP_HOURS = 6      # ERA5 data time step

# Visualization
PLOT_HEIGHT = 500
# ========================

# 1. Convert ssrd to W/m²
df['ssrd_wm2'] = df['ssrd'] / (TIME_STEP_HOURS * 3600)

# 2. Focus on peak production hours
df['hour'] = df['time'].dt.hour
peak_hours_mask = (df['hour'] >= PEAK_HOURS[0]) & (df['hour'] < PEAK_HOURS[1])
df_peak = df[peak_hours_mask].copy()

# 3. Find significant low irradiance events
if 'ssrdc' in df_peak.columns:
    df_peak['clear_sky_ratio'] = df_peak['ssrd'] / df_peak['ssrdc'].clip(lower=1)
    threshold_metric = 'clear_sky_ratio'
    threshold_value = 0.3  # 30% of clear-sky
else:
    threshold_metric = 'ssrd_wm2'
    threshold_value = THRESHOLD

# Find consecutive low production days
daily_min = df_peak.groupby(df_peak['time'].dt.date)[threshold_metric].min().reset_index()
daily_min.columns = ['date', 'min_value']
daily_min['low_prod_day'] = daily_min['min_value'] < threshold_value

# Find events of consecutive days
events = []
current_event = None

for i, row in daily_min.iterrows():
    if row['low_prod_day']:
        if current_event is None:
            current_event = {'start': row['date'], 'count': 1}
        else:
            current_event['count'] += 1
    else:
        if current_event is not None:
            if current_event['count'] >= MIN_CONSECUTIVE_DAYS:
                events.append((
                    current_event['start'],
                    row['date'] - pd.Timedelta(days=1)
                ))
            current_event = None

# Handle case where event continues to end of data
if current_event is not None and current_event['count'] >= MIN_CONSECUTIVE_DAYS:
    events.append((
        current_event['start'],
        daily_min['date'].iloc[-1]
    ))

# 4. Create the plot
fig = go.Figure()

# Add full time series
fig.add_trace(go.Scatter(
    x=df['time'],
    y=df['ssrd_wm2'],
    mode='lines',
    name='Solar Radiation',
    line=dict(color='lightgray', width=1)
))

# Add peak hour points
fig.add_trace(go.Scatter(
    x=df_peak['time'],
    y=df_peak['ssrd_wm2'],
    mode='markers',
    name=f'Peak Hours ({PEAK_HOURS[0]}-{PEAK_HOURS[1]}h)',
    marker=dict(color='blue', size=4)
))

# Add threshold line
fig.add_hline(
    y=THRESHOLD,
    line_dash='dash',
    line_color='red',
    annotation_text=f'Threshold: {THRESHOLD} W/m²',
    annotation_position='top right'
)

# Add event rectangles
for start_date, end_date in events:
    start_dt = pd.Timestamp(start_date) if not isinstance(start_date, pd.Timestamp) else start_date
    end_dt = pd.Timestamp(end_date) + pd.Timedelta(days=1) if not isinstance(end_date, pd.Timestamp) else end_date + pd.Timedelta(days=1)
    
    event_data = df_peak[(df_peak['time'].dt.date >= start_date) & 
                        (df_peak['time'].dt.date <= end_date)]
    min_rad = event_data['ssrd_wm2'].min()
    duration_days = (end_date - start_date).days + 1
    
    fig.add_vrect(
        x0=start_dt,
        x1=end_dt,
        fillcolor='red',
        opacity=0.2,
        line_width=0,
        annotation_text=f"{duration_days}d<br>{min_rad:.0f} W/m²"
    )

# Update layout
fig.update_layout(
    title=f'Significant Low Solar Radiation Events (≥{MIN_CONSECUTIVE_DAYS} days)',
    xaxis_title='Time',
    yaxis_title='Solar Radiation (W/m²)',
    height=PLOT_HEIGHT,
    showlegend=True
)

# Add range slider
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(step="all")
        ])
    )
)

fig.show()

# Print event details
if events:
    print(f"Found {len(events)} significant low radiation periods:")
    for i, (start_date, end_date) in enumerate(events, 1):
        duration_days = (end_date - start_date).days + 1
        event_data = df_peak[(df_peak['time'].dt.date >= start_date) & 
                            (df_peak['time'].dt.date <= end_date)]
        avg_rad = event_data['ssrd_wm2'].mean()
        min_rad = event_data['ssrd_wm2'].min()
        
        print(f"\nEvent {i}:")
        print(f"  Period:    {start_date} to {end_date}")
        print(f"  Duration:  {duration_days} days")
        print(f"  Avg Radiation: {avg_rad:.1f} W/m²")
        print(f"  Min Radiation: {min_rad:.1f} W/m²")
else:
    print("No significant low radiation periods found.")

Found 8 significant low radiation periods:

Event 1:
  Period:    2021-02-01 to 2021-02-03
  Duration:  3 days
  Avg Radiation: 6.1 W/m²
  Min Radiation: 5.1 W/m²

Event 2:
  Period:    2021-12-24 to 2021-12-26
  Duration:  3 days
  Avg Radiation: 8.0 W/m²
  Min Radiation: 5.5 W/m²

Event 3:
  Period:    2022-04-04 to 2022-04-08
  Duration:  5 days
  Avg Radiation: 27.5 W/m²
  Min Radiation: 13.0 W/m²

Event 4:
  Period:    2022-11-28 to 2022-12-02
  Duration:  5 days
  Avg Radiation: 7.9 W/m²
  Min Radiation: 4.6 W/m²

Event 5:
  Period:    2023-11-16 to 2023-11-18
  Duration:  3 days
  Avg Radiation: 10.9 W/m²
  Min Radiation: 6.5 W/m²

Event 6:
  Period:    2023-12-19 to 2023-12-25
  Duration:  7 days
  Avg Radiation: 5.3 W/m²
  Min Radiation: 2.2 W/m²

Event 7:
  Period:    2024-02-06 to 2024-02-09
  Duration:  4 days
  Avg Radiation: 10.0 W/m²
  Min Radiation: 4.0 W/m²

Event 8:
  Period:    2024-12-15 to 2024-12-17
  Duration:  3 days
  Avg Radiation: 9.5 W/m²
  Min Radiation: 8.

## Heatwaves

In [ ]:

df['t2m_c'] = df['t2m'] - 273.15

events = extreme_events.find_heatwaves(df['t2m_c'], threshold=30, min_duration=3)
fig = px.line(df, x='time', y='t2m_c', title='Heatwaves')
for start, end in events:
    fig.add_vrect(x0=df['time'][start], x1=df['time'][end], fillcolor='orange', opacity=0.2, line_width=0)
fig.show()


## High Cloud Cover Events

In [ ]:
events = extreme_events.find_high_cloud_cover(df['tcc'], threshold=0.9, min_duration=3)
fig = px.line(df, x='time', y='tcc', title='High Cloud Cover Events')
for start, end in events:
    fig.add_vrect(x0=df['time'][start], x1=df['time'][end], fillcolor='blue', opacity=0.2, line_width=0)
fig.show()


In [ ]:

# Calculate wind speed magnitude: sqrt(u² + v²)
df['wind_speed'] = np.sqrt(df['u10']**2 + df['v10']**2)

# Assuming you have a DataFrame 'df' with 'wind_speed' and 'time' columns
lull_events = extreme_events.find_wind_lulls(
    df['wind_speed'], 
    threshold=2.0,     # m/s
    min_duration=6     # hours
)

storm_events = extreme_events.find_wind_storms(
    df['wind_speed'], 
    threshold=25.0,    # m/s
    min_duration=3     # hours
)

# Create an interactive plot
fig = extreme_events.plot_wind_events(
    wind_speed_series=df['wind_speed'],
    time_series=df['time'],
    lull_events=lull_events,
    storm_events=storm_events,
    lull_threshold=2.0,
    storm_threshold=25.0,
    title='Wind Speed Extremes Analysis'
)

# Show the plot
fig.show()